In [1]:
import pandas as pd

SPLITS = {s: pd.read_csv(f"../data/splits/{s}.csv") for s in ["train","val","test"]}
split_map = pd.concat([v.assign(split=k) for k, v in SPLITS.items()])

master = pd.read_csv("../data/splits/master.csv")

merged = master.merge(split_map, on="image_id", how="left", suffixes=("_master", "_map"))

# coalesce split: prefer master’s split, fallback to map’s split
if "split_master" in merged and "split_map" in merged:
    merged["split"] = merged["split_master"].fillna(merged["split_map"])
elif "split_master" in merged:
    merged = merged.rename(columns={"split_master": "split"})
elif "split_map" in merged:
    merged = merged.rename(columns={"split_map": "split"})
else:
    raise KeyError("No split column found after merge.")

v1 = merged[["image_id","source_dataset","rel_path","race_cat","gender_cat","age_cat","split"]]
v1.to_csv("../data/labels_v1.csv", index=False)
print("✅ Wrote ../data/labels_v1.csv with", len(v1), "rows")
print(v1["split"].value_counts(dropna=False))


✅ Wrote ../data/labels_v1.csv with 900 rows
split
train    630
val      135
test     135
Name: count, dtype: int64
